# Annotating the data
For the annotations of the sample I use the quantative content analysis (Lamnek 2005). Here three categories will be formed:
1. non-answer: The category encompasses every response where no reaction to the question occurs. Example: ""
2. evasive answer: This category is defined as reacting to the question in not or just partly answering the question. Example: "Sehr geehrter Herr W., haben Sie vielen Dank für Ihre Anfrage. Ich beteilige mich nicht länger am Portal abgeordnetenwatch.de. Um Ihre Frage dennoch zu beantworten, bitte ich um Mitteilung Ihrer E-Mail-Adresse an antje.tillmann@bundestag.de. Mit freundlichen Grüßen Antje Tillmann MdB"
3. answer: Every response which contains the answer to the questions in annotated in this category. Expample: "Sehr geehrter Herr Schellerich,die gesamte Fraktion DIE LINKE im Deutschen Bundestag wird dem ESM-Vertrag nicht zustimmen. Ich habe dies in meiner Rede vom 29.März im Bundestag auch versucht zu begründen. Mit freundlichen Grüßen Dr. Gysi"

The drawn sample will be mannualy annotated. Next the sample will be used to categorise the rest of the answers automatically.

In [25]:
# load libraries for data manipulation
import pandas as pd
import re
import regex
import numpy as np

# ML: Train/test splits, cross validation,
# gridsearch
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
)

# load libraries for tokenization
import nltk
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
#nltk.download("stopwords")
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# load libraries for text cleaning
import spacy
import ufal.udpipe
from gensim.models import KeyedVectors, Phrases
from gensim.models.phrases import Phraser
from ufal.udpipe import Model, Pipeline
import conllu

# Supervised text classification
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import joblib
#import eli5



## Preprocessing

In [2]:
# load data
sample_df = pd.read_csv("./data/stratified_sample.csv")
# remove NaN for tokenizer to work
sample_df = sample_df.dropna(subset=["answer"])

The next step comprises the preprocessing of the data. All answers will be converted to lowercase, punctuation and other noise will be removed. Lowercasing each word has the advantage that there no two different writing styles of a word. I.e. "die" and "Die" are now recognized as the same word.

In [ ]:
# lower the answers to make the analysis easier
sample_df["answer"] = sample_df["answer"].str.lower()

# remove links and punctuation
sample_df["answer"] = sample_df["answer"].str.replace(r"\bhttps?://\S*|&\w+;|[\.,]", " ", regex=True)
sample_df["answer"] = sample_df["answer"].str.replace(r"\s+", " ", regex=True)

## Analyzing

In the next code chunk the sample data will be split into a training and test set. On the data of the training set the model will train and with the testing set the trained model will be tested. This step is necessary to avoid overfitting and ensure the quality of the results.

In [26]:
# split data into training and testing set with a testing set size of 20% of the data.
X_train, X_test, y_train, y_test = train_test_split(
    sample_df["answer"],
    sample_df["answer_encoded"],
    test_size=0.2,
    random_state=42
)

103 408
